In [210]:
!pip install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [211]:
pip install torch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [212]:
import torch

In [213]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")

In [214]:
pip install sentence_transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [215]:
from sentence_transformers import SentenceTransformer, util

In [216]:
pip install pandas

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [248]:
import json
from pprint import pprint
import pandas as pd

# Specify the path to your JSON file in Google Drive
file_path = "new-headline-train.json"

# Load data from the file
with open(file_path, "r") as file:
    dataset = json.load(file)

data = pd.DataFrame.from_dict(dataset)

data.head()


,id,input,profile
0,300,Generate a headline for the following article:...,[{'text': 'Homeowners sell their homes and buy...
1,301,Generate a headline for the following article:...,[{'text': 'This article summarizes the dysfunc...
2,302,Generate a headline for the following article:...,[{'text': 'Homeowners sell their homes and buy...
3,303,Generate a headline for the following article:...,[{'text': 'While the attractions and bathing s...
4,304,Generate a headline for the following article:...,[{'text': 'While the attractions and bathing s...


In [249]:
query_start = 'Generate a headline for the following article: '
query = dataset[0]['input'][len(query_start):]
print(query)

Your loans were made at the height of the housing bubble, and looked like a great deal at the time. By using a HELOC as a "piggyback" second mortgage, you were not required to make a down payment or to purchase mortgage insurance.


In [250]:
documents=[]
for doc in dataset[0]['profile']:
    documents.append(doc['text']) 
print(len(documents))

15


In [252]:
def paraphrase_t5(
    input_sentence,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=5,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=500
):
    input_ids = tokenizer(
        f'paraphrase: {input_sentence}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids
    
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    rephrased_versions = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return rephrased_versions

# Example usage
input_sentence = query
rephrased_versions = paraphrase_t5(input_sentence)


for i, rephrased_sentence in enumerate(rephrased_versions, start=1):
    print(f"Rephrased Version {i}: {rephrased_sentence}")
    print("-------------------------------------------")

Rephrased Version 1: The homes you borrowed money for were at the peak of the housing market and appeared attractive to investors. By using a HELOC as an optional secondary mortgage, you were not required to pay down or purchase mortgage insurance.
-------------------------------------------
Rephrased Version 2: During the height of this housing bubble, you made out-of-pocket loans that seemed like a good deal. By using your HELOC as part of your second mortgage, there was no need to pay remorse or purchase mortgage insurance.
-------------------------------------------
Rephrased Version 3: You had been granted loans during the height of the housing market, which appeared to be a great deal at the time. By using HGELOC as if it were putting on X amount towards your second mortgage, you avoided paying off the down payment and didn't need to purchase any mortgage insurance.
-------------------------------------------
Rephrased Version 4: At the height of the housing bubble, you had secur

In [253]:
#Load the model
model2 = SentenceTransformer('sentence-transformers/msmarco-distilbert-cos-v5')

In [254]:
#Encode query and documents
# query_emb = model2.encode(query)
doc_emb = model2.encode(documents)

In [255]:
# Initialize lists to store relevant documents and scores for each version
all_relevant_documents = []
all_relevant_scores = []

In [256]:
# Process each rephrased version and the original version
for i, input_version in enumerate([input_sentence] + rephrased_versions, start=1):
    # Print the input version
    print(f"\nProcessing Input Version {i}: {input_version}\n{'='*50}")
    
    # Retrieve relevant documents for the current version
    input_documents = []
    input_scores = []

    # Encode the current query version
    input_emb = model2.encode(input_version)

    # Compute similarity scores between the current query version and all documents
    similarity_scores = util.dot_score(input_emb, doc_emb)[0].cpu().tolist()

    doc_score_title_pairs = list(zip([doc['text'] for doc in dataset[0]['profile']],
                                     [doc['title'] for doc in dataset[0]['profile']],
                                     similarity_scores))

    # Sort by decreasing similarity score
    doc_score_title_pairs = sorted(doc_score_title_pairs, key=lambda x: x[2], reverse=True)
    
    # Calculate the number of documents to retrieve (top 25%)
    num_documents_to_retrieve = int(0.25 * len(doc_score_title_pairs))

    # Check if the number of documents to retrieve is greater than 15
    if num_documents_to_retrieve > 10:
        num_documents_to_retrieve = 10

    if num_documents_to_retrieve < 5:
        num_documents_to_retrieve = 5

    for text, title, score in doc_score_title_pairs[:num_documents_to_retrieve]:
        print(f"Original Score: {score}")
        print(f"Title: {title}")
        print(f"Document: {text}\n")
        
        # Store relevant document, title, and score for each interpretation
        input_documents.append({'text': text, 'title': title})
        input_scores.append(score)
        
        print("="*50)
        
    # Store relevant documents and scores for each version
    all_relevant_documents.append(input_documents)
    all_relevant_scores.append(input_scores)

    



Processing Input Version 1: Your loans were made at the height of the housing bubble, and looked like a great deal at the time. By using a HELOC as a "piggyback" second mortgage, you were not required to make a down payment or to purchase mortgage insurance.
Original Score: 0.43686643242836
Title: Conventional Versus FHA: Which Should You Choose?
Document: Don't jump to the conclusion that the better choice is the mortgage with the lower interest rate. FHAs carry a lower interest rate but largely because of their high insurance premiums, they usually (but not always) cost the borrower more.

Original Score: 0.3464226722717285
Title: Some Recent Letters on Downsizing and Mortgage Lender Mistakes
Document: In pricing loans used to purchase a home, lenders distinguish three possible uses of the property. Mortgages used to purchase a house that the purchasers intend to occupy as their primary residence get the best price.

Original Score: 0.3247550129890442
Title: Investing In a Larger Do

In [257]:
# Calculate average score for each document across different interpretations
average_scores = {}
for documents, scores in zip(all_relevant_documents, all_relevant_scores):
    for doc_dict, score in zip(documents, scores):
        if isinstance(doc_dict, dict):  # Check if it's a dictionary
            doc_text = doc_dict.get('text', '')  # Use 'get' to provide a default value if 'text' is not present
            doc_title = doc_dict.get('title', '')  # Use 'get' to provide a default value if 'title' is not present
            if doc_text:
                if doc_text not in average_scores:
                    average_scores[doc_text] = {'text': doc_text, 'title': doc_title, 'scores': []}
                average_scores[doc_text]['scores'].append(score)

# Calculate average score for each document
average_documents = [{'title': details['title'], 'text': details['text'], 'average_score': sum(details['scores']) / len(details['scores'])}for details in average_scores.values()]

# Sort documents based on average scores
sorted_documents = sorted(average_documents, key=lambda x: x['average_score'], reverse=True)

retrieval_docs = []

# Output top 10 documents based on average scores
print("\nTop  Documents After Average:")
for doc_dict in sorted_documents[:7]:
    new_doc_dict = {'title': doc_dict['title'], 'text': doc_dict['text']}
    retrieval_docs.append(new_doc_dict)
print(retrieval_docs)


Top  Documents After Average:
[{'title': 'Conventional Versus FHA: Which Should You Choose?', 'text': "Don't jump to the conclusion that the better choice is the mortgage with the lower interest rate. FHAs carry a lower interest rate but largely because of their high insurance premiums, they usually (but not always) cost the borrower more."}, {'title': 'Some Recent Letters on Downsizing and Mortgage Lender Mistakes', 'text': 'In pricing loans used to purchase a home, lenders distinguish three possible uses of the property. Mortgages used to purchase a house that the purchasers intend to occupy as their primary residence get the best price.'}, {'title': 'Investing In a Larger Down Payment: High Yields and No Risk', 'text': 'Consumers looking to purchase a home within the near future face many decisions, including how large a down payment to make. The down payment is the sale price (confirmed by a appraisal) less the loan amount. In most cases, home purchasers must have financial assets

In [258]:
r_d = ''
count = 1
for i in retrieval_docs:
  # print(i)
  r_d += str(count) + '. '
  r_d += 'The text is: ' + i['text'] + '\n'
  r_d += 'The title for the above text is: ' + i['title'] + '\n'
  count += 1
  # if count > 7:
  #   break

print(r_d)

1. The text is: Don't jump to the conclusion that the better choice is the mortgage with the lower interest rate. FHAs carry a lower interest rate but largely because of their high insurance premiums, they usually (but not always) cost the borrower more.
The title for the above text is: Conventional Versus FHA: Which Should You Choose?
2. The text is: In pricing loans used to purchase a home, lenders distinguish three possible uses of the property. Mortgages used to purchase a house that the purchasers intend to occupy as their primary residence get the best price.
The title for the above text is: Some Recent Letters on Downsizing and Mortgage Lender Mistakes
3. The text is: Consumers looking to purchase a home within the near future face many decisions, including how large a down payment to make. The down payment is the sale price (confirmed by a appraisal) less the loan amount. In most cases, home purchasers must have financial assets at least as large as the down payment they make.


PALM USING GOOGLE API

In [227]:
# pip install -U -q google-generativeai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [239]:
import google.generativeai as palm

import textwrap
import numpy as np
import pandas as pd

In [260]:
def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = textwrap.dedent("""You are a helpful and informative bot that answers questions using text from the reference passage included below.   
  Be sure to respond in a complete sentence, including all relevant background information. I'm providing you with some sample texts and title written by me for 7 texts examples.
  Based on the context provided, what's a relevant title for the following news article
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

In [261]:
passage = r_d

In [262]:
prompt = make_prompt(query, passage)
print(prompt)

You are a helpful and informative bot that answers questions using text from the reference passage included below.   
  Be sure to respond in a complete sentence, including all relevant background information. I'm providing you with some sample texts and title written by me for 7 texts examples.
  Based on the context provided, what's a relevant title for the following news article
  QUESTION: 'Your loans were made at the height of the housing bubble, and looked like a great deal at the time. By using a HELOC as a "piggyback" second mortgage, you were not required to make a down payment or to purchase mortgage insurance.'
  PASSAGE: '1. The text is: Dont jump to the conclusion that the better choice is the mortgage with the lower interest rate. FHAs carry a lower interest rate but largely because of their high insurance premiums, they usually (but not always) cost the borrower more. The title for the above text is: Conventional Versus FHA: Which Should You Choose? 2. The text is: In pr

In [263]:
text_models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]

text_model = text_models[0]

In [264]:
temperature = 0.5
answer = palm.generate_text(prompt=prompt,
                            model=text_model,
                            candidate_count=3,
                            temperature=temperature,
                            max_output_tokens=1000)

In [265]:
for i, candidate in enumerate(answer.candidates):
  print(f"Candidate {i}: {candidate['output']}\n")

Candidate 0: After March 2, Reverse Mortgage Borrowers Will Have to Prove They Are Not Deadbeats

Candidate 1: A relevant title for the following news article is: Beware of the HELOC "Piggyback" Second Mortgage.

Candidate 2: Borrowers with HELOCs and Piggyback Loans Face a Tough Choice

